In [1]:
import numpy as np
import cv2
from PIL import Image
import open3d as o3d
import json
import os
from tqdm import tqdm
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib import rcParams
from shapely.geometry import MultiPoint, box

# from pyquaternion.quaternion import Quaternion
from pyquaternion import Quaternion
import os.path as osp
from nuscenes import NuScenes

# Utils for Lidar and Radar
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.scripts.export_2d_annotations_as_json import get_2d_boxes, post_process_coords
from utils import *

from typing import Tuple, List, Dict, Union

%matplotlib inline

In [2]:
#usc = NuScenes(version='v1.0-mini', dataroot='/home/gus/Documents/AI_S/nuScenes/v1_0-mini', verbose=True)
nusc = NuScenes(version='v1.0-mini', dataroot='/home/sherlock/Documents/nuScenes/data', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.2 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [5]:
# Load the information from .txt file in JSON format
sample_path = os.getcwd()+'/dataset/samples/sample_841dd6709e9b4d7c9d6bf888f1fe6d7e_file.txt'

# This is a Dictionary of lists of dictionaries
data_json = load_file(sample_path)
data_json['instance'][0]

{'annotation_token': '4b8f390f86aa433cac6c4247f907565f',
 'sample_token': '841dd6709e9b4d7c9d6bf888f1fe6d7e',
 'camera_token': '39caaccc75dd4e38b8fe6f40e81f5621',
 'pointcloud_path': '/home/sherlock/Documents/3d_obj_detection/dataset/ann_points_lidar/pc_anno_4b8f390f86aa433cac6c4247f907565f.txt',
 'pcl_shape': [3, 794],
 'img_pc_path': '/home/sherlock/Documents/3d_obj_detection/dataset/ann_points_cam/img_pc_anno_4b8f390f86aa433cac6c4247f907565f.txt',
 'img_pcl_shape': [4, 794],
 'pcd_path': '/home/sherlock/Documents/3d_obj_detection/dataset/pcd_segmentation/pcd_segmentation_4b8f390f86aa433cac6c4247f907565f.pcd',
 'point_features_path': '/home/sherlock/Documents/3d_obj_detection/dataset/point_features/pcd_segmentation_4b8f390f86aa433cac6c4247f907565f.txt',
 'position_coord': [6.053757848521915, 4.1099025347326315, -0.899958760271085],
 'wlh_values': [1.772, 4.294, 1.597],
 'orientation_value': -177.91535813202847,
 'category': 'vehicle'}

In [4]:
from nuscenes.utils.geometry_utils import BoxVisibility

box_vis_level = BoxVisibility.ANY
    
sample_record = nusc.get('sample', data_json['instance'][0]['sample_token'])
cam = nusc.get('sample_data', data_json['instance'][0]['camera_token'])

_, boxes, camera_intrinsic = nusc.get_sample_data(sample_record['data'][cam['channel']], box_vis_level=box_vis_level,
                                                    selected_anntokens=[data_json['instance'][0]['annotation_token']])

https://github.com/nutonomy/nuscenes-devkit/blob/master/python-sdk/nuscenes/utils/data_classes.py#L572

https://github.com/nutonomy/nuscenes-devkit/blob/master/python-sdk/nuscenes/nuscenes.py#L1316

In [6]:
boxes[0]
camera_intrinsic

array([[1.26084744e+03, 0.00000000e+00, 8.07968245e+02],
       [0.00000000e+00, 1.26084744e+03, 4.95334427e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [7]:
boxes[0].corners()

array([[-0.52176648, -1.96608728, -1.91975839, -0.47543758,  1.96283816,
         0.51851735,  0.56484625,  2.00916706],
       [-0.18650868, -0.14021471,  1.45609862,  1.40980464, -0.24367553,
        -0.19738156,  1.39893177,  1.35263779],
       [ 4.37586692,  5.40142935,  5.39461757,  4.36905515,  7.87756762,
         8.90313004,  8.89631827,  7.87075584]])